In [ ]:
#wget "https://www.robots.ox.ac.uk/~vgg/data/fgvc-aircraft/archives/fgvc-aircraft-2013b.tar.gz"

In [ ]:
try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  print("We're running Colab")
if IN_COLAB:
  # Mount the Google Drive at mount
  mount='/content/gdrive'
  print("Colab: mounting Google drive on ", mount)

  drive.mount(mount)

  # Switch to the directory on the Google Drive that you want to use
  import os
  drive_root = mount + "/My Drive"
  #drive_root = mount + "/My Drive/maskrcnn-benchmark"
  # Create drive_root if it doesn't exist
  create_drive_root = True
  if create_drive_root:
    print("\nColab: making sure ", drive_root, " exists.")
    os.makedirs(drive_root, exist_ok=True)
  
  # Change to the directory
  print("\nColab: Changing directory to ", drive_root)
  %cd $drive_root

We're running Colab
Colab: mounting Google drive on  /content/gdrive
Mounted at /content/gdrive

Colab: making sure  /content/gdrive/My Drive  exists.

Colab: Changing directory to  /content/gdrive/My Drive
/content/gdrive/My Drive


In [ ]:
#!unzip  fgvc-aircraft-2013b.zip

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import gc

import cv2

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Dataset

In [ ]:
# Path
data_path = 'fgvc-aircraft-2013b/data'
img_path = 'fgvc-aircraft-2013b/data/images'

# IMG_SIZE is determined by EfficientNet model choice
IMG_SIZE = 240
NUM_CLASSES = 100

Image ID

In [ ]:
def read_id(filename):
  img = []
  f = open(filename, 'r', encoding='utf-8')
  lines = f.readlines()
  for line in lines:
    img.append(line.replace('\n', ''))
  f.close()
  return img

train_id = read_id(os.path.join(data_path, 'images_train.txt'))
val_id = read_id(os.path.join(data_path, 'images_val.txt'))
test_id = read_id(os.path.join(data_path, 'images_test.txt'))

print('Train, valid, test sizes: {} - {} - {}'.format(len(train_id), len(val_id), len(test_id)))

Train, valid, test sizes: 3334 - 3333 - 3333


1. Read images

In [ ]:
bbox = {}
f = open(os.path.join(data_path, 'images_box.txt'), 'r', encoding='utf-8')
lines = f.readlines()
for line in lines:
  arr = line.replace('\n', '').split(' ')
  bbox[arr[0]] = [int(arr[i]) for i in range(1, 5)]
f.close()

In [ ]:
def read_images(ids):
  images = []
  for id in ids:
    path = os.path.join(img_path, id + '.jpg')
    img = cv2.imread(path)
    x_min, y_min, x_max, y_max = bbox[id]
    img = img[y_min:y_max, x_min:x_max]
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    images.append(img)
  return np.array(images)

X_train = read_images(train_id)
X_val = read_images(val_id)
X_test = read_images(test_id)

In [ ]:
X_train.shape

(3334, 240, 240, 3)

2. Construct label

In [ ]:
def getFamilies(filename):
  families = {}
  f = open(os.path.join(data_path, filename), 'r', encoding='utf-8')
  lines = f.readlines()
  for line in lines:
    first_space = line.find(' ')
    id = line[:first_space]
    family = line[(first_space+1):].replace('\n', '')
    families[id] = family
  f.close()
  return families

train_families = getFamilies('images_family_train.txt')
val_families = getFamilies('images_family_val.txt')
test_families = getFamilies('images_family_test.txt')

# Number of families in train-valid-test set: 70 - 70 - 70

In [ ]:
def getVariants(filename):
  variants = {}
  f = open(os.path.join(data_path, filename), 'r', encoding='utf-8')
  lines = f.readlines()
  for line in lines:
    first_space = line.find(' ')
    id = line[:first_space]
    variant = line[(first_space+1):].replace('\n', '')
    variants[id] = variant
  f.close()
  return variants

train_variants = getVariants('images_variant_train.txt')
val_variants = getVariants('images_variant_val.txt')
test_variants = getVariants('images_variant_test.txt')

# Number of variants in train-valid-test set: 100 - 100 - 100

In [ ]:
def getManufacturers(filename):
  manufacturers = {}
  f = open(os.path.join(data_path, filename), 'r', encoding='utf-8')
  lines = f.readlines()
  for line in lines:
    first_space = line.find(' ')
    id = line[:first_space]
    manufacturer = line[(first_space+1):].replace('\n', '')
    manufacturers[id] = manufacturer
  f.close()
  return manufacturers

train_manufacturers = getManufacturers('images_manufacturer_train.txt')
val_manufacturers = getManufacturers('images_manufacturer_val.txt')
test_manufacturers = getManufacturers('images_manufacturer_test.txt')

# Number of manufacturers in train-valid-test set: 30 - 30 - 30

In [ ]:
def labels(ids, families, variants, manufacturers):
  f = []
  v = []
  m = []

  for id in ids:
    f.append(families[id])
    v.append(variants[id])
    m.append(manufacturers[id])

  df = pd.DataFrame({'Family': f, 'Variant': v, 'Manufacturer': m})
  return df

y_train = labels(train_id, train_families, train_variants, train_manufacturers)
y_val = labels(val_id, val_families, val_variants, val_manufacturers)
y_test = labels(test_id, test_families, test_variants, test_manufacturers)

print('Train labels size:', y_train.shape)
print('Valid labels size:', y_val.shape)
print('Test labels size:', y_test.shape)

Train labels size: (3334, 3)
Valid labels size: (3333, 3)
Test labels size: (3333, 3)


In [ ]:
# Label encoder
def encoding_map(filename):
  mEncoder = {}
  i = 0
  f = open(os.path.join(data_path, filename), 'r', encoding='utf-8')
  lines = f.readlines()
  for line in lines:
    mEncoder[line.replace('\n', '')] = int(i)
    i += 1
  f.close()
  return mEncoder

family_encoder = encoding_map('families.txt')
variant_encoder = encoding_map('variants.txt')
manufacturer_encoder = encoding_map('manufacturers.txt')

print('Number of families:', len(family_encoder))
print('Number of variants:', len(variant_encoder))
print('Number of manufacturers:', len(manufacturer_encoder))

Number of families: 70
Number of variants: 100
Number of manufacturers: 30


In [ ]:
variant_encoder

{'707-320': 0,
 '727-200': 1,
 '737-200': 2,
 '737-300': 3,
 '737-400': 4,
 '737-500': 5,
 '737-600': 6,
 '737-700': 7,
 '737-800': 8,
 '737-900': 9,
 '747-100': 10,
 '747-200': 11,
 '747-300': 12,
 '747-400': 13,
 '757-200': 14,
 '757-300': 15,
 '767-200': 16,
 '767-300': 17,
 '767-400': 18,
 '777-200': 19,
 '777-300': 20,
 'A300B4': 21,
 'A310': 22,
 'A318': 23,
 'A319': 24,
 'A320': 25,
 'A321': 26,
 'A330-200': 27,
 'A330-300': 28,
 'A340-200': 29,
 'A340-300': 30,
 'A340-500': 31,
 'A340-600': 32,
 'A380': 33,
 'ATR-42': 34,
 'ATR-72': 35,
 'An-12': 36,
 'BAE 146-200': 37,
 'BAE 146-300': 38,
 'BAE-125': 39,
 'Beechcraft 1900': 40,
 'Boeing 717': 41,
 'C-130': 42,
 'C-47': 43,
 'CRJ-200': 44,
 'CRJ-700': 45,
 'CRJ-900': 46,
 'Cessna 172': 47,
 'Cessna 208': 48,
 'Cessna 525': 49,
 'Cessna 560': 50,
 'Challenger 600': 51,
 'DC-10': 52,
 'DC-3': 53,
 'DC-6': 54,
 'DC-8': 55,
 'DC-9-30': 56,
 'DH-82': 57,
 'DHC-1': 58,
 'DHC-6': 59,
 'DHC-8-100': 60,
 'DHC-8-300': 61,
 'DR-400': 62,


In [ ]:
# l_m=list(train_manufacturers.keys())

# # =  list(train_manufacturers.values())
# train_manufacturers[train_id[0]]
# train_variants[train_id[0]]
# my_d={}
def CountFrequency(my_list):
     
    # Creating an empty dictionary
    freq = {}
    for items in my_list:
        freq[items] = my_list.count(items)
     
    for key, value in freq.items():
        print ("% s : % d"%(key, value))
l_m=list(train_manufacturers.values())
l_v=list(train_variants.values())

CountFrequency(l_m)

Boeing :  733
Airbus :  434
ATR :  66
Antonov :  34
British Aerospace :  133
Beechcraft :  67
Lockheed Corporation :  68
Douglas Aircraft Company :  133
Canadair :  134
Cessna :  133
McDonnell Douglas :  232
de Havilland :  167
Robin :  33
Dornier :  34
Embraer :  233
Eurofighter :  33
Lockheed Martin :  34
Dassault Aviation :  67
Fokker :  100
Bombardier Aerospace :  33
Gulfstream Aerospace :  67
Ilyushin :  33
Fairchild :  33
Piper :  33
Cirrus Aircraft :  33
Saab :  67
Supermarine :  33
Panavia :  34
Tupolev :  66
Yakovlev :  34


In [ ]:
l_m=list(train_manufacturers.values())
l_v=list(train_variants.values())

# =  list(train_manufacturers.values())

dict_lv={}
for i in range(len(l_m)):
  if l_m[i] not in dict_lv.keys():
    dict_lv[l_m[i]]=[l_v[i]]
  elif l_v[i] not in dict_lv[l_m[i]]:
    dict_lv[l_m[i]].append(l_v[i])
print(dict_lv)


{'Boeing': ['707-320', '727-200', '737-200', '737-300', '737-400', '737-500', '737-600', '737-700', '737-800', '737-900', '747-100', '747-200', '747-300', '747-400', '757-200', '757-300', '767-200', '767-300', '767-400', '777-200', '777-300', 'Boeing 717'], 'Airbus': ['A300B4', 'A310', 'A318', 'A319', 'A320', 'A321', 'A330-200', 'A330-300', 'A340-200', 'A340-300', 'A340-500', 'A340-600', 'A380'], 'ATR': ['ATR-42', 'ATR-72'], 'Antonov': ['An-12'], 'British Aerospace': ['BAE 146-200', 'BAE 146-300', 'BAE-125', 'Hawk T1'], 'Beechcraft': ['Beechcraft 1900', 'Model B200'], 'Lockheed Corporation': ['C-130', 'L-1011'], 'Douglas Aircraft Company': ['C-47', 'DC-3', 'DC-6', 'DC-8'], 'Canadair': ['CRJ-200', 'CRJ-700', 'CRJ-900', 'Challenger 600'], 'Cessna': ['Cessna 172', 'Cessna 208', 'Cessna 525', 'Cessna 560'], 'McDonnell Douglas': ['DC-10', 'DC-9-30', 'F/A-18', 'MD-11', 'MD-80', 'MD-87', 'MD-90'], 'de Havilland': ['DH-82', 'DHC-1', 'DHC-6', 'DHC-8-100', 'DHC-8-300'], 'Robin': ['DR-400'], 'Dor

In [ ]:
label_lv=[]
list_dict= list( dict_lv.keys())
for i in range(len(list_dict )):
  label_lv.append(len(dict_lv[list_dict[i]]))
len(label_lv)
label_lv

[22,
 13,
 2,
 1,
 4,
 2,
 2,
 4,
 4,
 4,
 7,
 5,
 1,
 1,
 7,
 1,
 1,
 2,
 3,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1]

In [ ]:
list_dict= list( dict_lv.keys())

LABEL=[[variant_encoder.get(dict_lv[list_dict[i]][j]) for j in range(len(dict_lv[list_dict[i]]))] for i in range(len(list_dict ))]

len(LABEL)  

30

In [ ]:
y_val.shape

(3333, 3)

In [ ]:
def numerical_labels(ids, families, variants, manufacturers):
  f = []
  v = []
  m = []

  for id in ids:
    f.append(family_encoder[families[id]])
    v.append(variant_encoder[variants[id]])
    m.append(manufacturer_encoder[manufacturers[id]])

  df = pd.DataFrame({'Family': f, 'Variant': v, 'Manufacturer': m})
  return df

pos_y_train = numerical_labels(train_id, train_families, train_variants, train_manufacturers)
pos_y_val = numerical_labels(val_id, val_families, val_variants, val_manufacturers)
pos_y_test = numerical_labels(test_id, test_families, test_variants, test_manufacturers)
pos_a_train = numerical_labels(train_id, train_families, train_variants, train_manufacturers)
pos_a_val = numerical_labels(val_id, val_families, val_variants, val_manufacturers)
pos_a_test = numerical_labels(test_id, test_families, test_variants, test_manufacturers)

print('Train labels size:', pos_y_train.shape)
print('Valid labels size:', pos_y_val.shape)
print('Test labels size:', pos_y_test.shape)

Train labels size: (3334, 3)
Valid labels size: (3333, 3)
Test labels size: (3333, 3)


In [ ]:
# from tensorflow.keras.utils import to_categorical

# def OneHotEncoder(label):
#   f = to_categorical(label['Family'], num_classes=family_class, dtype='int')
#   # f = to_categorical(label['Family'], num_classes=variant_class, dtype='int') 
#   v = to_categorical(label['Variant'], num_classes=variant_class, dtype='int')             
#   # m = to_categorical(label['Manufacturer'], num_classes=manufacturer_class, dtype='int')
#   m = to_categorical(label['Manufacturer'], num_classes=variant_class, dtype='int')

#   lf = f.tolist()
#   lv = v.tolist()
#   lm = m.tolist()

#   arr = []
#   for i in range(label.shape[0]):
#     tmp = []
#     tmp.append(lf[i])
#     tmp.append(lv[i])
#     tmp.append(lm[i])
#     arr.append(tmp)

#   label['Family'] = lf
#   label['Variant'] = lv
#   label['Manufacturer'] = lm
  
#   # return label
#   return np.array(arr)

# y_train = OneHotEncoder(pos_y_train)
# y_val = OneHotEncoder(pos_y_val)
# y_test = OneHotEncoder(pos_y_test)


In [ ]:
family_class = 70
variant_class = 100
manufacturer_class = 30

# One-hot encoder
from tensorflow.keras.utils import to_categorical

def OneHotEncoder(label):
  f = to_categorical(label['Family'], num_classes=family_class, dtype='int')
  v = to_categorical(label['Variant'], num_classes=variant_class, dtype='int')             
  m = to_categorical(label['Manufacturer'], num_classes=manufacturer_class, dtype='int')

  lf = f.tolist()
  lv = v.tolist()
  lm = m.tolist()

  label['Family'] = lf
  label['Variant'] = lv
  label['Manufacturer'] = lm
  
  return label

y_train = OneHotEncoder(pos_y_train)
y_val = OneHotEncoder(pos_y_val)
y_test = OneHotEncoder(pos_y_test)

a_train = OneHotEncoder(pos_a_train)
a_val = OneHotEncoder(pos_a_val)
a_test = OneHotEncoder(pos_a_test)

In [ ]:
y_train.drop(columns=['Manufacturer','Family'], inplace=True)
y_val.drop(columns=[ 'Manufacturer','Family'], inplace=True)
y_test.drop(columns=[ 'Manufacturer','Family'], inplace=True)

# np.array(y_train)

In [ ]:
a_train.drop(columns=['Family','Variant'], inplace=True)
a_val.drop(columns=['Family','Variant'], inplace=True)
a_test.drop(columns=['Family','Variant'], inplace=True)

In [ ]:

def convert_NP(label):
  arr = []
  for i in range(label['Variant'].shape[0]):
    arr.append(label['Variant'].loc[i])
  arr = np.array(arr)
  return arr
y_train= convert_NP(y_train)
y_val= convert_NP(y_val)
y_test= convert_NP(y_test)

# test= convert_NP(a_train)
# a_val= convert_NP(a_val)
# a_test= convert_NP(a_test)

In [ ]:
def convert_NP(label):
  arr = []
  for i in range(label['Manufacturer'].shape[0]):
    arr.append(label['Manufacturer'].loc[i])
  arr = np.array(arr)
  return arr
a_train= convert_NP(a_train)
a_val= convert_NP(a_val)
a_test= convert_NP(a_test)

In [ ]:
m_pred=np.array([])

for num in label_lv:
  if m_pred.shape[0]==0:
    m_pred=np.sum(y_train[:,:num], axis=1)
  else:
   m_pred=np.append(m_pred,np.sum(y_train[:,:num],axis=1))
   
# np.sum(y_train[:,:2], axis=1)
# y_train[:,:2]
m_pred=m_pred.reshape((30, m_pred.shape[0]//30)).T
# m_pred=m_pred.reshape(( m_pred.shape[0]//30,30))


test=np.sum(y_train[:,:label_lv[1]], axis=1)

a=m_pred[:,1]- test
sum(a)
# a=np.array([1,2,3,4,5,6])
# a.reshape(3,2)

a=np.append(a_train[0],np.zeros(70))
for i in range(1,a_train.shape[0]):
  
  a=np.row_stack((a,np.append(a_train[i],np.zeros(70))))

b=np.append(a_val[0],np.zeros(70))
for i in range(1,a_val.shape[0]):
  
  b=np.row_stack((b,np.append(a_val[i],np.zeros(70))))

x= np.zeros((3334,2,100))
for i in range ( y_train.shape[0]):
  x[i]= np.row_stack((y_train[i],a[i]))
y_train=x

y= np.zeros((3333,2,100))
for i in range ( y_val.shape[0]):
  y[i]= np.row_stack((y_val[i],b[i]))
y_val=y

In [ ]:
print(y_train.shape)
print(y_val.shape)


(3334, 2, 100)
(3333, 2, 100)


In [ ]:
def format_label(label):
  def get_key(my_dict, val):
    for key, value in my_dict.items():
         if val == value:
             return key
 
    return "key doesn't exist"
  
  f = get_key(family_encoder, label['Family'])
  v = get_key(variant_encoder, label['Variant'])
  m = get_key(manufacturer_encoder, label['Manufacturer'])
  
  # return f"Family: {f} ; Variant: {v} ; Manufacturer: {m}"
  return f, v, m

def decode_label(label):
  def get_key(my_dict, val):
    for key, value in my_dict.items():
         if val == value:
             return key
 
    return "key doesn't exist"
  
  f = get_key(family_encoder, np.argmax(label[0]))
  v = get_key(variant_encoder, np.argmax(label[1]))
  m = get_key(manufacturer_encoder, np.argmax(label[2]))
  
  return f"Family: {f} ; Variant: {v} ; Manufacturer: {m}"

# Setup

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    print("Device:", tpu.master())
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError:
    print("Not connected to a TPU runtime. Using CPU/GPU strategy")
    strategy = tf.distribute.MirroredStrategy()

Not connected to a TPU runtime. Using CPU/GPU strategy
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [ ]:
gc.collect()

52

# Data Augumentation

In [ ]:
img_augmentation = Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

In [ ]:
# ! git clone https://github.com/yiyixuxu/polyloss-pytorch.git
# %cd polyloss-pytorch
y_train[:,0,:]

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
# def poly1_cross_entropy(epsilon):
#     def _poly1_cross_entropy(y_true, y_pred):
#         # pt, CE, and Poly1 have shape [batch].
#         pt = tf.reduce_sum(float(y_true) * float(tf.nn.softmax(y_pred)), axis=-1)
#         CE = tf.nn.softmax_cross_entropy_with_logits(y_true, y_pred)
#         Poly1 = CE + epsilon * (1 - pt)
#         #loss = tf.reduce_mean(Poly1)
#         return Poly1
#     return _poly1_cross_entropy

def poly1_cross_entropy(epsilon ,label=LABEL ):
    def _poly1_cross_entropy(y_true, y_pred,alpha=0.1):
        # pt, CE, and Poly1 have shape [batch].
        num_classes = y_true[:,0,:].get_shape().as_list()[-1]
        smooth_labels = float(y_true[:,0,:]) * float(1-alpha) + alpha/float(num_classes)
        one_minus_pt = tf.reduce_sum(
        float(smooth_labels) * float(1 - tf.nn.softmax(y_pred)), axis=-1)
        CE_loss = tf.keras.losses.CategoricalCrossentropy(
          from_logits=True, label_smoothing=alpha, reduction='none')
        CE = CE_loss(y_true[:,0,:], y_pred)
        Poly1_V = CE + epsilon * one_minus_pt

########################################
        m_pred=tf.constant([])
        x= y_pred


        for i in range(0,len(LABEL)):
          if i==0:
            m_pred=tf.reduce_sum( [x[:,index]   for index in LABEL[i]],axis=0)
          else:
            m_pred=tf.concat([m_pred,tf.reduce_sum( [x[:,index]   for index in LABEL[i]],axis=0)],axis=0)

        m_pred=tf.reshape(m_pred,[30, -1])
        m_pred=tf.transpose(m_pred)

        smooth_labels = float(y_true[:,1,:30]) * float(1-alpha) + alpha/float(num_classes)
        one_minus_pt = tf.reduce_sum(
        float(smooth_labels) * float(1 - tf.nn.softmax(m_pred)), axis=-1)
        CE_loss = tf.keras.losses.CategoricalCrossentropy(
          from_logits=True, label_smoothing=alpha, reduction='none')
        CE = CE_loss(y_true[:,1,:30], m_pred)
        Poly1_M = CE + epsilon * one_minus_pt
        # Poly1_M=0
        #print(y_true.get_shape())
        return Poly1_V + 0.5*Poly1_M
    return _poly1_cross_entropy

# def poly1_cross_entropy(epsilon):
#     def _poly1_cross_entropy(y_true, y_pred,alpha=0.1):
#         # pt, CE, and Poly1 have shape [batch].
#         num_classes = y_true.get_shape().as_list()[-1]
#         smooth_labels = float(y_true) * float(1-alpha) + alpha/float(num_classes)
#         one_minus_pt = tf.reduce_sum(
#         float(smooth_labels) * float(1 - tf.nn.softmax(y_pred)), axis=-1)
#         CE_loss = tf.keras.losses.CategoricalCrossentropy(
#           from_logits=True, label_smoothing=alpha, reduction='none')
#         CE = CE_loss(y_true, y_pred)
#         Poly1 = CE + epsilon * one_minus_pt
#         print(y_true.get_shape())
#         return Poly1
#     return _poly1_cross_entropy

def poly1_focal_loss(gamma=2.0, epsilon=1.0, alpha=0.25):
    def _poly1_focal_loss(y_true, y_pred):
        p = tf.math.sigmoid(y_pred)
        ce_loss = tf.nn.sigmoid_cross_entropy_with_logits(y_true, y_pred)
        pt = y_true * p + (1 - y_true) * (1 - p)
        FL = ce_loss * ((1 - pt) ** gamma)
        
        if alpha >= 0:
            alpha_t = alpha * y_true + (1 - alpha) * (1 - y_true)
            FL = alpha_t * FL
        Poly1 = FL + epsilon * tf.math.pow(1 - pt, gamma + 1)
        loss = tf.reduce_mean(Poly1)
        return loss
    return _poly1_focal_loss

In [ ]:
import keras.backend as K
m = tf.keras.metrics.Accuracy()
def my_metric_fn(y_true, y_pred):
    
    # score= 0
    # for i in range(y_true.get_shape().as_list()[0]):
    #    if tf.math.argmax(y_true[i,0])==tf.math.argmax(y_pred[i]):
    #      score+=1
    true=  (tf.math.argmax(y_true[:,0,:], axis=1)+1)
    pred=  (tf.math.argmax(y_pred, axis=1)+1)
    
    m.update_state(true, pred)
    
    return m.result()
      # return tf.cast(tf.math.equal(y_true[:,0,:], y_pred), tf.float32)

# accuracy_metric = tf.keras.metrics.MeanMetricWrapper(fn=my_metric_fn)
def kullback_leibler_divergence(y_true, y_pred):
    Y= y_true[:,0,:]
    y_true[:,0,:].assign(K.clip(Y, K.epsilon(), 1))
    y_pred = K.clip(Y, K.epsilon(), 1)
    return K.sum(Y * K.log(Y / y_pred), axis=-1)

In [ ]:
x= tf.convert_to_tensor(y_train[:,0,:])
print(x)
      
a_pred=tf.constant([])

for i in range(0,len(LABEL)):
  if i==0:
    a_pred=tf.reduce_sum( [x[:,index]   for index in LABEL[i]],axis=0)
  else:
    a_pred=tf.concat([a_pred,tf.reduce_sum( [x[:,index]  for index in LABEL[i]],axis=0)],axis=0)
  #print(a_pred)

a_pred=tf.reshape(a_pred,[30, -1])
a_pred=tf.transpose(a_pred)
a_pred
test=np.sum(y_train[:,0,LABEL[2]], axis=1)

a=a_pred[:,2]- test

sum(a)
LABEL# if tf.math.argmax(x[0,0]) == tf.math.argmax(x[0,0]):
#   print(0)
# xr = (tf.math.argmax(x[:,0,:], axis=1)+1)
# # a= tf.convert_to_tensor(0,1) 
# # a
# m = tf.keras.metrics.Accuracy()
# m.update_state(xr, xr)
# m.result()

tf.Tensor(
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]], shape=(3334, 100), dtype=float64)


[[0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  41],
 [21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33],
 [34, 35],
 [36],
 [37, 38, 39, 82],
 [40, 90],
 [42, 84],
 [43, 53, 54, 55],
 [44, 45, 46, 51],
 [47, 48, 49, 50],
 [52, 56, 73, 85, 86, 87, 88],
 [57, 58, 59, 60, 61],
 [62],
 [63],
 [64, 65, 66, 67, 68, 69, 70],
 [71],
 [72],
 [74, 75],
 [76, 77, 78],
 [79],
 [80, 81],
 [83],
 [89],
 [91],
 [92],
 [93, 94],
 [95],
 [96],
 [97, 98],
 [99]]

In [ ]:
LABEL

[[0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  41],
 [21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33],
 [34, 35],
 [36],
 [37, 38, 39, 82],
 [40, 90],
 [42, 84],
 [43, 53, 54, 55],
 [44, 45, 46, 51],
 [47, 48, 49, 50],
 [52, 56, 73, 85, 86, 87, 88],
 [57, 58, 59, 60, 61],
 [62],
 [63],
 [64, 65, 66, 67, 68, 69, 70],
 [71],
 [72],
 [74, 75],
 [76, 77, 78],
 [79],
 [80, 81],
 [83],
 [89],
 [91],
 [92],
 [93, 94],
 [95],
 [96],
 [97, 98],
 [99]]

# Train model

In [ ]:
from tensorflow.keras.applications import EfficientNetB1

def build_model(num_classes):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = img_augmentation(inputs)
    #x = inputs
    model = EfficientNetB1(include_top=False, input_tensor=x, weights='imagenet')

    # Freeze the pretrained weights ImageNet
    #model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    print(x)
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    # x = layers.Dense(200, activation='relu', name='dense_1')(x)
    # x = layers.Reshape((2,100))(x)
    # outputs= softmax(x,axis=2)
    outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)
    print(x)
    print(outputs.shape)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    model.compile(
        #optimizer=optimizer, loss=poly1_cross_entropy(epsilon=0), metrics=["accuracy"]
        #optimizer=optimizer, loss=poly1_cross_entropy(epsilon=1,label=LABEL)
         optimizer=optimizer, loss=kullback_leibler_divergence
        #, metrics=[my_metric_fn]                                                    ############## ACC
        #optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"]
    )
    
    return model

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                filepath='best_b1_poly_e=2.h5',
                                save_weights_only=True,
                                monitor='val_loss',
                                mode='min',
                                save_best_only=True,
                                verbose=1
                            )
model_early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, min_delta=0, verbose=0, mode='min')

In [ ]:
# del model

# K.clear_session()
# gc.collect()
# del model

NameError: ignored

In [ ]:
#model.summary()



############## CHAY O DAY LEN #####################


with strategy.scope():
    model = build_model(num_classes=y_val.shape[-1])
#model.load_weights('b7_244.h5')

epochs = 60
hist= model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val), verbose=2,
               callbacks=[model_checkpoint_callback],
                          # , model_early_stopping_callback],
                batch_size=64)
# model.fit(train_generator, validation_data=val_generator, verbose=2, epochs=epochs)

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)



Epoch 1: val_loss improved from inf to 6.98535, saving model to best_b1_poly_e=2.h5
53/53 - 101s - loss: 7.7230 - val_loss: 6.9853 - 101s/epoch - 2s/step
Epoch 2/60

Epoch 2: val_loss improved from 6.98535 to 6.47359, saving model to best_b1_poly_e=2.h5
53/53 - 61s - loss: 6.1080 - val_loss: 6.4736 - 61s/epoch - 1s/step
Epoch 3/60

Epoch 3: val_loss improved from 6.47359 to 5.86317, saving model to best_b1_poly_e=2.h5
53/53 - 62s - loss: 5.4326 - val_loss: 5.8632 - 62s/epoch - 1s/step
Epoch 4/60

Epoch 4: val_loss improved from 5.86317 to 5.61311, saving model to best_b1_poly_e=2.h5
53/53 - 62s - loss: 4.9913 - val_loss: 5.6131 - 62s/epoch - 1s/step
Epoch 5/60

Epoch 5: val_loss improved from 5.61311 to 5.43842, saving model to best_b1_poly_e=2.h5
53/53 - 63s - loss: 4.7850 - val_loss: 5.4384 - 63s/epoch - 1s/step
Epoch 6/60

Epoch 6: val_loss improved from 5.43842 to 5.35058, saving model to best_b1_poly_e=2.h5
53/53 - 63s - loss: 4.5970 - val_loss: 5.3506 - 63s/epoch - 1s/step
Epoch

In [ ]:
with strategy.scope():
    model = build_model(num_classes=y_val.shape[-1])
#model.load_weights('b7_244.h5')

epochs = 60  # @param {type: "slider", min:8, max:80}
hist= model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val), verbose=2,
               callbacks=[model_checkpoint_callback],
                          # , model_early_stopping_callback],
                batch_size=64)
# model.fit(train_generator, validation_data=val_generator, verbose=2, epochs=epochs)

KerasTensor(type_spec=TensorSpec(shape=(None, 1280), dtype=tf.float32, name=None), name='batch_normalization_4/batchnorm/add_1:0', description="created by layer 'batch_normalization_4'")
KerasTensor(type_spec=TensorSpec(shape=(None, 1280), dtype=tf.float32, name=None), name='top_dropout/Identity:0', description="created by layer 'top_dropout'")
(None, 100)
Epoch 1/60
INFO:tensorflow:Error reported to Coordinator: in user code:

    File "<ipython-input-54-c85a36031b9c>", line 20, in kullback_leibler_divergence  *
        y_true[:,0,:].assign(K.clip(Y, K.epsilon(), 1))

    AttributeError: 'Tensor' object has no attribute 'assign'
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/training/coordinator.py", line 293, in stop_on_exception
    yield
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/distribute/mirrored_run.py", line 342, in run
    self.main_result = self.main_fn(*self.main_args, **self.main_kwargs)
  File "/us

AttributeError: ignored

In [ ]:
model.save('b1_poly_e=1_1_05.h5')
model.save_weights('b1_poly_e=1_1_05_weights.h5')
np.save('b1_poly_e=1_1_05.npy',hist.history)
#history=np.load('my_history.npy',allow_pickle='TRUE').item()


In [ ]:
model.save_weights('b1_CE_weights.h5')

In [ ]:
with strategy.scope():
    model = build_model(num_classes=100)
#model.load_weights('b4_poly.h5')

epochs = 43  # @param {type: "slider", min:8, max:80}
hist= model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val), verbose=2, batch_size=64)
# model.fit(train_generator, validation_data=val_generator, verbose=2, epochs=epochs)

# Evaluation

In [ ]:
def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()


plot_hist(hist)

In [ ]:
history=np.load('b1_poly_smooth_history.npy',allow_pickle='TRUE').item()


In [ ]:
def plot_hist(hist):
    plt.plot(hist["accuracy"])
    plt.plot(hist["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()


plot_hist(history)

# **TEST**

In [ ]:
with strategy.scope():
    model = build_model(num_classes=100)
model.load_weights('best_b1_poly_e=1_1_05.h5')


KerasTensor(type_spec=TensorSpec(shape=(None, 1280), dtype=tf.float32, name=None), name='batch_normalization_5/batchnorm/add_1:0', description="created by layer 'batch_normalization_5'")
KerasTensor(type_spec=TensorSpec(shape=(None, 1280), dtype=tf.float32, name=None), name='top_dropout/Identity:0', description="created by layer 'top_dropout'")
(None, 100)


In [ ]:

############## HAM TEST #####################



pred= model.predict(X_test,batch_size=64)



In [ ]:
score=0
addition=0
for i in range(pred.shape[0]):
  if np.argmax(pred[i])!=np.argmax(y_test[i]):
    sorted_index_array = np.argsort(pred[i])
  
    # sorted array
    n =4
    sorted_array = pred[i][sorted_index_array[-n : ]]
    # pred[0]
    
      
    # we are using negative
    # indexing concept
      
    # find n largest value
    rslt = sorted_array[-n : ]
    print(sorted_index_array[-n:])
    print(np.argmax(y_test[i]))
    if np.argmax(y_test[i]) in sorted_index_array[-n:]:
      addition+=1

    #print(rslt)
  else:
    score+=1
score/=3333
print(score)
addition/3333


[10 26  0 56]
0
[56 97  0 55]
0
[52 84 55 56]
0
[29 55  0 14]
0
[14 10 84 55]
0
[14 55 10 52]
0
[11 14 10 55]
0
[ 8 54  0 55]
0
[ 0 22 10 56]
0
[ 1 27 86 55]
0
[ 1  9 98 41]
1
[ 1 40 52 85]
1
[39 98 78 21]
1
[84  1 72 97]
1
[ 1 40 84  0]
1
[98  1 99 84]
1
[ 0  1 86 13]
1
[21 85  1 84]
1
[40  0 14 56]
1
[ 1 51 85 99]
1
[10 55 56 84]
1
[35 22 84  2]
1
[82 19  1 43]
1
[52  2 22 56]
2
[ 0 54 74 21]
2
[61 57 22  5]
2
[17  3  0 11]
2
[ 3  7  2 56]
2
[ 2 19 78  0]
2
[95 58 56 53]
2
[14 21  2 60]
2
[4 3 2 5]
2
[ 5 16  2 23]
2
[16  5  3 21]
3
[9 8 3 7]
3
[ 9 27  3 21]
3
[6 3 4 5]
3
[52  4  5 21]
3
[60 56  3  5]
3
[94 42  4  5]
3
[ 6  1 19  5]
3
[ 3 35 45  5]
3
[65  5 21  4]
3
[35 55  5  4]
3
[21  4  3  5]
3
[23 64 14 56]
3
[21  3  4  5]
3
[ 6 21  5  4]
3
[8 3 5 7]
3
[64  3 55  7]
3
[6 4 5 7]
3
[67  7  3  4]
3
[ 4  3 21  5]
3
[66  8  9  7]
3
[32 98 60  6]
3
[8 3 7 9]
4
[ 4 21  5  3]
4
[ 5 21  4  3]
4
[39  5  4  3]
4
[8 7 9 3]
4
[ 3  4 22 21]
4
[14 22  9 21]
4
[8 3 4 9]
4
[ 4 55  3  7]
4
[ 7 56 2

0.1551155115511551

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
p= np.zeros(pred.shape[0])
t= np.zeros(y_test.shape[0])
for i in range(pred.shape[0]):
    p[i]=np.argmax(pred[i])
    t[i]= np.argmax(y_test[i])

m=confusion_matrix(p,t)
m.trace()/m.shape[0]


24.77

In [ ]:
pred[0]

In [ ]:
import tensorflow as tf
from tensorflow.keras.activations import softmax
x=tf.constant([[-10, -5, 0.0, 5, 10]], dtype = tf.float32)
#x=tf.keras.layers.Flatten()(x)
#print(x)
x=layers.Dense(10, activation='relu', name='dense_1')(x)
#print(x)
#print(x.numpy())
x = layers.Reshape((2,5))(x)
# arr= np.array([])
# for i in range(x.shape[0]):
#     tmp=layers.Dense(x.shape[2], activation='softmax', name='dense_2')( tf.convert_to_tensor(x[i]))
#     arr=np.append(arr,tmp.numpy())
# # arr= arr[:-2]
# print(np.array([arr]))
# y=np.array([arr])
# # x[0,1]
# x=tf.convert_to_tensor(y)
x = softmax(x,axis=2)
x





In [ ]:
x[0,1]

In [ ]:
ex1 = tf.convert_to_tensor([[2,-1,1],[-1,2,0],[1,0,2]],dtype=tf.float32)

tf.linalg.eigvals(ex1)